# Introduction to Lakeflow in Databricks

In this notebook we'll introduce Lakeflow by setting up Casper's two main Jobs & Pipelines: `dlt_order_items` and `Orders In Progress`

The `dlt_order_items` pipeline processes the raw order events produced by Casper's using the medallion architecture:

  1. Bronze: Read JSON formed events that arrive on a Volume and write them into a bronze landing table `all_events`
  2. Silver: Explore the `order_created` events into 1 item per row (order_items) silver table
  3. Gold: Create multiple gold streaming tables for downstream consumption in SQL analysis, Dashboards, Genie spaces etc.

The `Orders In Progress` streaming job:

  1. Reads from Bronze table of `dlt_order_items` above
  2. Aggregates all events per order and keeps them in the stream state
  3. Writes out all the events of an order once `delivered` event comes in. 

### Setup

We need to initialize data and catalogs before we begin creating our pipeline.

In [0]:
from utils.utils import (
    setup_catalog_and_volume,
    copy_raw_data_to_volume,
    drop_gk_demo_catalog,
    initialize_dimension_tables
)

# Drop existing catalog/volume/table if you need to start fresh
drop_gk_demo_catalog(spark)

## 1. Setup the catalog and volume
setup_catalog_and_volume(spark)

## 2. Copy the raw data to the volume
copy_raw_data_to_volume()

## 3. Initialize the static dimension tables
initialize_dimension_tables(spark)


#### `order_items_dlt` Declarative Pipeline

The code for the pipeline is prepared in the `./pipelines/order_items_dlt` directory. To initialize this code as a declarative pipeline, we need to go to `Jobs & Pipelines` in the main navigation bar and click `Create` and then select `ETL Pipeline`

![](./images/lakeflow/1.png)


In the new page you'll need to:

1. Name the pipeline in the top left corner (**order_items**)
2. Select `gk_demo` catalog and create a new schema `lakeflow` for all the pipeline assets 
3. Select `Add existing assets` and select the folder `./pipelines/order_items_dlt/` in this repository for both paths


![](./images/lakeflow/2.png)
![](./images/lakeflow/3.png)


Once you add the pipeline code you'll see this page that provides:

1. All pipeline assets (code) in the left hand pane
2. Tab based editor in the center pane
3. Table & Performance results in the bottom pane 
4. A visual dependency graph in the right hand pane

![](./images/lakeflow/4.png)


Click `Run Pipeline` to start the pipeline and watch the panes populate with the results

![](./images/lakeflow/5.png)


We've now completed the main declarative pipeline for Casper's. 

- **Bronze layer** – ingest the raw JSON events exactly as they arrive.
- **Silver layer**– clean / normalize: explode the items array inside each order and add useful columns.
- **Gold layer** – aggregate the cleaned data into business-ready tables (order-, item-, brand-, and location-level metrics).

By default, everything runs in manual trigger mode to save resources, but you can change it to continuous streaming mode if you like.

The code for the transformations in each stage is mostly self explantory but here are some notes on each stage:

##### 0. Bronze – all_events
- Uses Auto Loader (cloudFiles) to watch the /raw_data directory for new JSON files—one file per event.
- No transformation—just lands the raw payload in Delta.

Result: a streaming DLT table called all_events.

##### 1. Silver – dlt_order_items
- Filter only order_created events.
- Cast the string timestamp field ts to event_ts (true TIMESTAMP).
- Parse the body JSON string into a typed struct (body_obj) with your body_schema.
- Explode the items array so each line now represents one item within an order.
- Compute `extended_price` = price × qty.
- Derive `order_day` for partitioning.
- Select tidy columns to output.

The table is partitioned by `order_day` for fast date-based queries.

##### 2-A. Gold – dlt_order_header
Aggregates the Silver table to one row per order:

- `order_revenue` – sum of all items’ extended_price.
- `total_qty` – total units in the order.
- `total_items` – count of distinct line items.
- `brands_in_order` – unique list of brand_ids bought.

Useful for dashboards that care about order-level revenue rather than individual line items.

##### 2-B. Gold – dlt_item_sales_day
Daily item performance:

- Groups by `item_id` (plus its menu/category/brand) and the order_day.
- Emits `units_sold` and gross_revenue for each item per calendar day.

Partitioned by day to keep each day’s data together.

##### 2-C. Gold – dlt_brand_sales_day
Brand-level daily roll-up with two streaming-specific features:

Watermark (`withWatermark("order_ts", "3 hours"))` – tells Spark to discard data arriving more than 3 h late, keeping state manageable.

`approx_count_distinct` – HyperLogLog sketch to estimate unique orders per brand without heavy state storage.

Outputs: estimated order count, items sold, revenue for each brand on each day.

##### 2-D-1. Gold – dlt_location_sales_hourly
Hourly KPIs per ghost-kitchen location:

- Apply the same 3-hour watermark.
- Round each `order_ts` down to the hour with `date_trunc("hour", …)`.
- Aggregate by location and that truncated hour timestamp.
- Collect approximate order count and revenue.

Partitioned by `hour_ts` to speed queries that slice by time.

#### Notes:
- Layered medallion architecture keeps raw and transformed data separate, simplifying debugging and schema evolution.
- Streaming DLT ensures each layer updates automatically as new order files land.
- Watermark + HLL choices strike a balance between accuracy and scalability in a perpetual stream.


### `Orders In Progress` streaming job

The `Orders In Progress` streaming job is responsible for aggregating events (by `order_id`) as they arrive and writing them out as a single record when an order has completed (the `delivered` event has arrived).

The code for this streaming job is defined using a notebook and is located at `./notebooks/Orders In Progress.ipynb`.

Starting the `Jobs & Pipelines` tab in the main left navigation bar, click `Create > Job`

![](./images/lakeflow/job1.png)


Give the job a new title (**Orders In Progress Stream**) and click `Notebook` under `Add your first task`

![](./images/lakeflow/job2.png)


Give your task a name (`main`) and specify the source of the job as the notebook path as shown in the image. Finally select `Create Task` and `Run Now` to start the first run of this streaming job.

**Note:** the streaming job is written using the `AvailableNow` trigger, so it will execute once and stop. This is done to save resources in the Free edition. To process new data (in future examples), you'll need to manually run the job again.

![](./images/lakeflow/job3.png)


Once the job finishes running you can query the state using the query in the next cell. This leverages [Spark Streaming's State Reader API.](https://docs.databricks.com/aws/en/structured-streaming/read-state)

This same query will be used in the `Apps` demo to surface orders in progress to a Databricks App.

(**Note** blocked on ES-1538976)

In [0]:
%sql

SELECT 
    key.order_id as order_id,
    collect_list(list_element) as events 
FROM read_statestore(
    "/Volumes/gk_demo/default/checkpoints/orders_in_progress",
    stateVarName => 'events' ) 
GROUP BY key.order_id